In [1]:
import json
from pathlib import Path

import folium
import opera_utils
import pandas as pd

In [16]:
def explore(gdf, column, cmap="RdBu_r", vmax=240, vmin=30):  # noqa: D103
    m = folium.Map()

    # Create separate layers for each orbit pass type
    ascending = gdf[gdf["orbit_pass"] == "ASCENDING"].copy()
    descending = gdf[gdf["orbit_pass"] == "DESCENDING"].copy()
    # Sort by longitude (west to east) so eastern frames are on top
    # Use the centroid of each geometry to determine its longitude
    ascending.loc[:, "center_x"] = ascending.geometry.to_crs("EPSG:3857").centroid.x
    ascending = ascending.sort_values(by="center_x")
    descending.loc[:, "center_x"] = descending.geometry.to_crs("EPSG:3857").centroid.x
    descending = descending.sort_values(by="center_x")

    # Add layers with different colors
    ascending.explore(
        m=m,
        cmap=cmap,
        column=column,
        name="ASCENDING",
        vmin=vmin,
        vmax=vmax,
        style_kwds={"fillOpacity": 0.7},
    )

    descending.explore(
        m=m,
        cmap=cmap,
        column=column,
        name="DESCENDING",
        vmin=vmin,
        vmax=vmax,
        style_kwds={"fillOpacity": 0.7},
        legend=False,
    )

    folium.LayerControl().add_to(m)
    return m

In [17]:
gdf_frames = opera_utils.get_frame_geodataframe()
overrides = json.loads(
    Path("../opera-disp-s1-algorithm-parameters-overrides-2025-08-28.json").read_text()
)

records = [
    {"frame_id": int(key), "data": json.dumps(value)}
    for key, value in overrides.items()
]
df_overrides = pd.DataFrame(records).set_index("frame_id")

In [18]:
df_overrides.head()

,data
frame_id,
1380,"{""recommended_similarity_threshold"": 0.75, ""re..."
1381,"{""recommended_similarity_threshold"": 0.75, ""re..."
1382,"{""recommended_similarity_threshold"": 0.75, ""re..."
1383,"{""recommended_similarity_threshold"": 0.75, ""re..."
1384,"{""recommended_similarity_threshold"": 0.75, ""re..."


In [19]:
gdf_merged = pd.merge(
    gdf_frames, df_overrides, left_index=True, right_index=True, how="left"
)

In [20]:
explore(gdf_merged.dropna(), column="data")